**12 Days of Demos**
# 🎅 North Pole Agent Tools (MCP) 🎄

Santa's operation is becoming increasingly complex. To help the elves (and Santa) make data-driven decisions, we are building a suite of **AI Agents**. But these agents need access to real-time data about the workshop, reindeer, and gift requests.

This notebook demonstrates how to use the **Model Context Protocol (MCP)** to give agents access to Unity Catalog SQL Functions. Instead of just chatting, our agents can now **query the data** to answer questions like:

* **Workshop Status**: How much glitter do we have left? Which elf team is performing best?
* **Reindeer Telemetry**: Is Rudolph tired? What is the average flight altitude?
* **Logistics**: Which delivery routes are impacted by weather?
* **Gift Trends**: What are the most popular toys this year?

### 🦌 Step 1: Configuration

Before you begin: Update the configuration below to match your environment.

The default values point to the demo dataset, but you can customize:
* **Catalog name** - Your Unity Catalog catalog
* **Schema name** - Schema where your raw data are stored
* **Volume Name** - Place where files should be stored prior to loading

👇 **Update the cell below with your values, then run it!**

In [ ]:
TARGET_CATALOG = "12daysofdemos"
TARGET_SCHEMA = "raw_data"
TARGET_VOLUME = "raw_data_volume"

# Set variables for the functions below
catalog_name = TARGET_CATALOG
schema_name = TARGET_SCHEMA

In [ ]:
%run "../00-init/load-data"

### 🛠️ Step 2: Define Agent Tools (SQL Functions)

To give our AI agents the ability to "see" into our operations, we need to define **tools**. In the Model Context Protocol (MCP), tools are often implemented as functions that the agent can call.

We will create a suite of **Unity Catalog SQL Functions** that encapsulate our business logic. These functions act as the API for our agents, allowing them to:
*   **Check inventory levels** 🏭
*   **Monitor reindeer health** 🦌
*   **Analyze gift requests** 🎁
*   **Optimize delivery routes** 🚚

*By defining these as SQL functions, we ensure that our agents always use approved, secure, and consistent logic!*

👇 **Run the cells below to register these functions in Unity Catalog!**

### 🏭 Workshop Production Metrics
*Tracking material usage to ensure we don't run out of glitter!* ✨

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_workshop_material_totals')()
RETURNS TABLE(total_plastic_kg DOUBLE, total_wood_kg DOUBLE, total_metal_kg DOUBLE, total_fabric_kg DOUBLE)
COMMENT 'Returns total usage of all materials in the workshop.'
RETURN (
  SELECT 
    SUM(materials_plastic_kg) as total_plastic_kg,
    SUM(materials_wood_kg) as total_wood_kg,
    SUM(materials_metal_kg) as total_metal_kg,
    SUM(materials_fabric_kg) as total_fabric_kg
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
);
""")

### 🧝 Elf Team Performance
*Monitoring quality scores to find out which team deserves extra hot cocoa.* ☕

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_team_quality_metrics')(team_ids STRING)
RETURNS TABLE(team_id STRING, avg_paint_quality DOUBLE, avg_assembly_score DOUBLE, total_defects BIGINT)
COMMENT 'Returns quality metrics for specific elf teams. Input: JSON Array. Example: "["Team_B", "Team_C"]"'
RETURN (
  SELECT 
    elf_team_id as team_id,
    AVG(paint_quality) as avg_paint_quality,
    AVG(assembly_score) as avg_assembly_score,
    SUM(defect_count) as total_defects
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
  WHERE array_contains(from_json(team_ids, 'ARRAY<STRING>'), elf_team_id)
  GROUP BY elf_team_id
);
""")

### 📍 Workshop Location Stats
*Comparing efficiency across our North, South, East, and West workshops.* 🧭

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_workshop_location_metrics')(locations STRING)
RETURNS TABLE(workshop_location STRING, avg_production_time DOUBLE, avg_carbon_footprint DOUBLE)
COMMENT 'Returns performance metrics for specific workshop locations. Input: JSON Array. Example: "["North_Workshop", "South_Workshop"]"'
RETURN (
  SELECT 
    workshop_location,
    AVG(production_time_minutes) as avg_production_time,
    AVG(carbon_footprint_kg) as avg_carbon_footprint
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
  WHERE array_contains(from_json(locations, 'ARRAY<STRING>'), workshop_location)
  GROUP BY workshop_location
);
""")

### 🧸 Toy Defect Rates
*Ensuring every toy brings joy, not disappointment.* 🔧

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_toy_defect_rate')(categories STRING)
RETURNS TABLE(toy_category STRING, defect_rate DOUBLE, total_produced BIGINT)
COMMENT 'Returns the percentage of toys with defects for given categories. Input: JSON Array. Example: "["VEHICLES", "TOYS"]"'
RETURN (
  SELECT 
    toy_category,
    CAST(SUM(CASE WHEN defect_count > 0 THEN 1 ELSE 0 END) AS DOUBLE) / COUNT(*) as defect_rate,
    COUNT(*) as total_produced
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
  WHERE array_contains(from_json(categories, 'ARRAY<STRING>'), toy_category)
  GROUP BY toy_category
);
""")

### ⏰ Shift Performance
*Analyzing productivity during the Morning, Afternoon, and Night shifts.* 🌙

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_shift_performance')(shifts STRING)
RETURNS TABLE(shift STRING, avg_quality_tier STRING, avg_production_time DOUBLE)
COMMENT 'Returns average production time and most common quality tier for specific shifts. Input: JSON Array. Example: "["MORNING", "NIGHT"]"'
RETURN (
  SELECT 
    shift,
    mode(quality_tier) as avg_quality_tier,
    AVG(production_time_minutes) as avg_production_time
  FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production')
  WHERE array_contains(from_json(shifts, 'ARRAY<STRING>'), shift)
  GROUP BY shift
);
""")

### 📬 Delivery Preferences
*Chimney, door, or window? Knowing how to deliver is half the battle!* 🏠

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_gift_requests_by_delivery_pref')(preferences STRING)
RETURNS TABLE(delivery_preference STRING, request_count BIGINT, avg_urgency DOUBLE)
COMMENT 'Returns count of requests and average urgency for delivery preferences. Input: JSON Array. Example: "["chimney", "door"]"'
RETURN (
  SELECT 
    delivery_preference,
    COUNT(*) as request_count,
    AVG(urgency_level) as avg_urgency
  FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests')
  WHERE array_contains(from_json(preferences, 'ARRAY<STRING>'), delivery_preference)
  GROUP BY delivery_preference
);
""")

### 🎁 Popular Gifts
*What's trending this year? (Besides peace on earth).* 📈

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_popular_gifts_by_category')(categories STRING)
RETURNS TABLE(item_name STRING, request_count BIGINT)
COMMENT 'Returns the top 5 most requested gifts in specific categories. Input: JSON Array. Example: "["TOYS", "SPORTS"]"'
RETURN (
  SELECT 
    gift_item.item_name,
    COUNT(*) as request_count
  FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests')
  LATERAL VIEW explode(from_json(extracted_gifts_json, 'ARRAY<STRUCT<item_name: STRING, category: STRING, confidence_score: DOUBLE>>')) AS gift_item
  WHERE array_contains(from_json(categories, 'ARRAY<STRING>'), primary_gift_category)
  GROUP BY 1
  ORDER BY 2 DESC
  LIMIT 5
);
""")

### 🚨 High Urgency Requests
*Prioritizing the most critical wishes.* 🆘

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_high_urgency_requests')(min_urgency INT)
RETURNS TABLE(request_id STRING, child_id STRING, urgency_level INT, country STRING)
COMMENT 'Returns a list of gift requests with urgency level greater than or equal to the specified value. Example: 5'
RETURN (
  SELECT 
    request_id,
    child_id,
    urgency_level,
    country
  FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests')
  WHERE urgency_level >= min_urgency
);
""")

### 🌍 Requests by Country
*Mapping out global joy distribution.* 🗺️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_gift_requests_by_country')(countries STRING)
RETURNS TABLE(country STRING, total_requests BIGINT, avg_sentiment DOUBLE)
COMMENT 'Returns total requests and average sentiment score for specific countries. Input: JSON Array. Example: "["China", "USA"]"'
RETURN (
  SELECT 
    country,
    COUNT(*) as total_requests,
    AVG(sentiment_score) as avg_sentiment
  FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests')
  WHERE array_contains(from_json(countries, 'ARRAY<STRING>'), country)
  GROUP BY country
);
""")

### 🦌 Reindeer Flight Metrics
*Checking speed and altitude stats for our antlered aviators.* ✈️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_reindeer_flight_metrics')(reindeer_names STRING)
RETURNS TABLE(reindeer_name STRING, avg_speed_mph DOUBLE, avg_altitude_feet DOUBLE, max_heart_rate INT)
COMMENT 'Returns average flight metrics for specific reindeer. Input: JSON Array. Example: "["Rudolph", "Dasher"]"'
RETURN (
  SELECT 
    reindeer_name,
    AVG(speed_mph) as avg_speed_mph,
    AVG(altitude_feet) as avg_altitude_feet,
    MAX(heart_rate) as max_heart_rate
  FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry')
  WHERE array_contains(from_json(reindeer_names, 'ARRAY<STRING>'), reindeer_name) AND flight_status = 'FLYING'
  GROUP BY reindeer_name
);
""")

### 🛸 Live Flight Status
*Who is currently in the air?* 📡

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_flying_reindeer_status')()
RETURNS TABLE(reindeer_name STRING, current_speed DOUBLE, current_altitude DOUBLE)
COMMENT 'Returns a list of reindeer currently in flight status.'
RETURN (
  SELECT 
    reindeer_name,
    speed_mph as current_speed,
    altitude_feet as current_altitude
  FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry')
  WHERE flight_status = 'FLYING'
  ORDER BY timestamp DESC
  LIMIT 9
);
""")

### 🥕 Low Energy Reindeer
*Identifying who needs a carrot break.* 🥕

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_low_energy_reindeer')(threshold DOUBLE)
RETURNS TABLE(reindeer_name STRING, energy_efficiency DOUBLE, timestamp TIMESTAMP)
COMMENT 'Returns reindeer with energy efficiency below a certain threshold. Example: 0.85'
RETURN (
  SELECT 
    reindeer_name,
    energy_efficiency,
    timestamp
  FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry')
  WHERE energy_efficiency < threshold
);
""")

### 🐾 Hoof Pressure Analysis
*Ensuring soft landings on every roof.* 🏠

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_average_hoof_pressure')()
RETURNS TABLE(avg_pressure_fl DOUBLE, avg_pressure_fr DOUBLE, avg_pressure_rl DOUBLE, avg_pressure_rr DOUBLE)
COMMENT 'Returns average hoof pressure for all reindeer across all four limbs.'
RETURN (
  SELECT 
    AVG(hoof_pressure_fl) as avg_pressure_fl,
    AVG(hoof_pressure_fr) as avg_pressure_fr,
    AVG(hoof_pressure_rl) as avg_pressure_rl,
    AVG(hoof_pressure_rr) as avg_pressure_rr
  FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry')
);
""")

### 🚚 Route Metrics
*Optimizing the path from A to B (or North Pole to Everywhere).* 🛣️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_delivery_route_metrics')(start_city STRING, end_city STRING)
RETURNS TABLE(start_city STRING, end_city STRING, avg_distance_km DOUBLE, avg_delivery_window DOUBLE)
COMMENT 'Returns average distance and delivery window for a specific route segment. Example: "Tokyo", "London"'
RETURN (
  SELECT 
    start_city,
    end_city,
    AVG(distance_km) as avg_distance_km,
    AVG(delivery_window_minutes) as avg_delivery_window
  FROM IDENTIFIER('{catalog_name}.{schema_name}.delivery_optimization')
  WHERE start_city = start_city AND end_city = end_city
  GROUP BY start_city, end_city
);
""")

### ❄️ Weather Impact
*How much does a blizzard slow us down?* 🌨️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_weather_impact_on_delivery')(weather_conditions STRING)
RETURNS TABLE(weather_condition STRING, avg_delay_minutes DOUBLE, count_routes BIGINT)
COMMENT 'Returns average delay minutes caused by specific weather conditions. Input: JSON Array. Example: "["HEAVY_SNOW", "FOG"]"'
RETURN (
  SELECT 
    weather_condition,
    AVG(weather_delay_minutes) as avg_delay_minutes,
    COUNT(*) as count_routes
  FROM IDENTIFIER('{catalog_name}.{schema_name}.delivery_optimization')
  WHERE array_contains(from_json(weather_conditions, 'ARRAY<STRING>'), weather_condition)
  GROUP BY weather_condition
);
""")

### 🧱 Chimney Accessibility
*Calculating the "Santa-fit" factor for each city.* 🎅

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_chimney_accessibility')(city_names STRING)
RETURNS TABLE(city_name STRING, total_households BIGINT, chimneys_available BIGINT, accessibility_rate DOUBLE)
COMMENT 'Returns chimney availability statistics for given cities. Input: JSON Array. Example: "["London", "Paris"]"'
RETURN (
  SELECT 
    start_city as city_name,
    SUM(households_count) as total_households,
    SUM(chimneys_available) as chimneys_available,
    SUM(chimneys_available) / SUM(households_count) as accessibility_rate
  FROM IDENTIFIER('{catalog_name}.{schema_name}.delivery_optimization')
  WHERE array_contains(from_json(city_names, 'ARRAY<STRING>'), start_city)
  GROUP BY start_city
);
""")

### 📜 Naughty or Nice Score
*The most important metric of all.* ⚖️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_child_naughty_nice_score')(child_ids STRING)
RETURNS TABLE(child_id STRING, current_score DOUBLE, gift_tier STRING)
COMMENT 'Returns the current running score and gift tier recommendation for specific children. Input: JSON Array. Example: "["CH_IQ_30379"]"'
RETURN (
  SELECT 
    child_id,
    running_score as current_score,
    gift_tier_recommendation as gift_tier
  FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics')
  WHERE array_contains(from_json(child_ids, 'ARRAY<STRING>'), child_id)
  ORDER BY timestamp DESC
  LIMIT 1
);
""")

### 🎭 Behavior Event Analysis
*Tracking kindness, sharing, and... tantrums.* 📊

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_behavior_events_by_type')(event_types STRING)
RETURNS TABLE(event_type STRING, event_count BIGINT, avg_impact DOUBLE)
COMMENT 'Returns statistics for specific types of behavioral events. Input: JSON Array. Example: "["SHARING", "KINDNESS_ACT"]"'
RETURN (
  SELECT 
    event_type,
    COUNT(*) as event_count,
    AVG(impact_score) as avg_impact
  FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics')
  WHERE array_contains(from_json(event_types, 'ARRAY<STRING>'), event_type)
  GROUP BY event_type
);
""")

### 🌟 Top Nice Children
*The gold standard of good behavior.* 🏆

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_top_nice_children')()
RETURNS TABLE(child_id STRING, running_score DOUBLE, country STRING)
COMMENT 'Returns the top N children with the highest nice scores.'
RETURN (
  SELECT 
    child_id,
    MAX(running_score) as running_score,
    first(location_country) as country
  FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics')
  GROUP BY child_id
  ORDER BY running_score DESC
  LIMIT 25
);
""")

### ⚫ Coal Warning List
*It's not too late to turn it around!* ⚠️

In [ ]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_coal_warning_list')()
RETURNS TABLE(child_id STRING, running_score DOUBLE, warning_date TIMESTAMP)
COMMENT 'Returns a list of children currently in the COAL_WARNING tier.'
RETURN (
  SELECT 
    child_id,
    running_score,
    timestamp as warning_date
  FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics')
  WHERE gift_tier_recommendation = 'COAL_WARNING'
  ORDER BY timestamp DESC
);
""")

### 🔍 Lookup Functions
*Helper functions to help agents discover valid input values.* 🔎

In [ ]:
# -------------------------------------------------------------------------
# Lookup Functions (Grounding)
# -------------------------------------------------------------------------

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_elf_teams')()
RETURNS TABLE(elf_team_id STRING)
COMMENT 'Returns a list of all valid elf team IDs.'
RETURN (SELECT DISTINCT elf_team_id FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production') ORDER BY 1);
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_workshop_locations')()
RETURNS TABLE(workshop_location STRING)
COMMENT 'Returns a list of all valid workshop locations.'
RETURN (SELECT DISTINCT workshop_location FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production') ORDER BY 1);
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_toy_categories')()
RETURNS TABLE(toy_category STRING)
COMMENT 'Returns a list of all valid toy categories.'
RETURN (SELECT DISTINCT toy_category FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production') ORDER BY 1);
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_shifts')()
RETURNS TABLE(shift STRING)
COMMENT 'Returns a list of all valid shifts.'
RETURN (SELECT DISTINCT shift FROM IDENTIFIER('{catalog_name}.{schema_name}.workshop_production') ORDER BY 1);
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_delivery_preferences')()
RETURNS TABLE(delivery_preference STRING)
COMMENT 'Returns a list of all valid delivery preferences.'
RETURN (SELECT DISTINCT delivery_preference FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests') ORDER BY 1);
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_countries')()
RETURNS TABLE(country STRING)
COMMENT 'Returns a list of all valid countries.'
RETURN (SELECT DISTINCT country FROM IDENTIFIER('{catalog_name}.{schema_name}.gift_requests') ORDER BY 1);
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_reindeer_names')()
RETURNS TABLE(reindeer_name STRING)
COMMENT 'Returns a list of all valid reindeer names.'
RETURN (SELECT DISTINCT reindeer_name FROM IDENTIFIER('{catalog_name}.{schema_name}.reindeer_telemetry') ORDER BY 1);
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_weather_conditions')()
RETURNS TABLE(weather_condition STRING)
COMMENT 'Returns a list of all valid weather conditions.'
RETURN (SELECT DISTINCT weather_condition FROM IDENTIFIER('{catalog_name}.{schema_name}.delivery_optimization') ORDER BY 1);
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION IDENTIFIER('{catalog_name}.{schema_name}.get_valid_behavior_event_types')()
RETURNS TABLE(event_type STRING)
COMMENT 'Returns a list of all valid behavior event types.'
RETURN (SELECT DISTINCT event_type FROM IDENTIFIER('{catalog_name}.{schema_name}.behavioral_analytics') ORDER BY 1);
""")